In [2]:
import pandas as pd
import matplotlib as plt
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf


In [3]:
data = pd.read_csv("review_data.csv")
data.head(10)

text  star  \
0                                    Anathi Khanyile     5   
1  Tony bahut funny hai Hill climbing racing my f...     5   
2                                            Teturwu     1   
3  Hoooooooooooyaaaaaaaaa what a game hoooooooooo...     5   
4                                  This game is nice     5   
5                                        Rahulyadavo     5   
6  First thing is that I like this game, it's fan...     4   
7                                          Very taty     3   
8                                               good     5   
9                                   I LIKE THIS GAME     5   

                     app_id reviewDate  
0  com.fingersoft.hillclimb   18/03/21  
1  com.fingersoft.hillclimb   18/03/21  
2  com.fingersoft.hillclimb   18/03/21  
3  com.fingersoft.hillclimb   18/03/21  
4  com.fingersoft.hillclimb   18/03/21  
5  com.fingersoft.hillclimb   18/03/21  
6  com.fingersoft.hillclimb   18/03/21  
7  com.fingersoft.hillclimb   18/03/21  
8  com.fingersoft.hillclimb   18/03/21  
9  com.fingersoft.hillclimb   18/03/21

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        29999 non-null  object
 1   star        30000 non-null  int64 
 2   app_id      30000 non-null  object
 3   reviewDate  30000 non-null  object
dtypes: int64(1), object(3)
memory usage: 937.6+ KB


In [5]:
data.isnull().sum()

text          1
star          0
app_id        0
reviewDate    0
dtype: int64

In [6]:
data.duplicated().sum()

2187

In [7]:
data.drop_duplicates(inplace=True)

In [8]:
data.duplicated().sum()

0

In [9]:
data.dropna(inplace=True)

In [10]:
data.isnull().sum()

text          0
star          0
app_id        0
reviewDate    0
dtype: int64

In [11]:
data.shape

(27812, 4)

In [12]:
data = data[['text','star']]

In [13]:
review = data[(data['star']>3)|(data['star']<=2)]

In [14]:
review['star'].value_counts()

5    18249
1     4470
4     2800
2      872
Name: star, dtype: int64

Observation : we found number of people 18249 who gave rating = 5 and number of people 2800 who gave rating = 4, number of people 4470 who gave rating = 1 and number of people 872 who gave rating = 2

In [15]:
import warnings
warnings.filterwarnings("ignore")
def partition(x):
  if x>3:
    return 1
  else:
    return 0
a = review['star']
b = a.map(partition)
review['star'] = b
print("Number of data points in out data",review.shape)
review.head()

Number of data points in out data (26391, 2)


text  star
0                                    Anathi Khanyile     1
1  Tony bahut funny hai Hill climbing racing my f...     1
2                                            Teturwu     0
3  Hoooooooooooyaaaaaaaaa what a game hoooooooooo...     1
4                                  This game is nice     1

In [16]:
review['star'].value_counts()

1    21049
0     5342
Name: star, dtype: int64

In [17]:
review.shape

(26391, 2)

In [18]:
x = review.drop(['star'],axis=1)
y = review['star']

**Import Pre-trained model and tokenizer**

In [19]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 60.2 MB/s 
     |████████████████████████████████| 895 kB 56.5 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 6.5 MB 45.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [20]:
from transformers import BertTokenizer

#Load Bert Tokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Splitting the dataset in train and test

In [21]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y,random_state=0)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(21112, 1)
(5279, 1)
(21112,)
(5279,)


In [22]:
max_seq_length = 55 # we assumed

In [23]:
def mask_and_segment(text_array,max_seq_length):
  x_mask=[]
  x_segment=[]
  x_tokens=[]
  for t in text_array:
    token = tokenizer.tokenize(t)
    if len(token)==(max_seq_length)-2:
      tokens = ['[CLS]',*token,'[SEP']
    elif len(token)>(max_seq_length)-2:
      tokens = ['[CLS]',*token[:max_seq_length-2],'[SEP]']
    else:
      pad_len = (max_seq_length-2)-len(token) #amount of adding to be done
      pads = ['[PAD']*pad_len
      tokens = ['[CLS]',*token,*pads,'[SEP]']
    mask = [1 if i!='[PAD]' else 0 for i in tokens]
    segment = [0]*max_seq_length
    tokens = tokenizer.convert_tokens_to_ids(tokens)
    x_mask.append(mask)
    x_segment.append(segment)
    x_tokens.append(tokens)
  return (np.array(x_mask),np.array(x_segment),np.array(x_tokens))

In [24]:
x_train_mask,x_train_segment,x_train_tokens = mask_and_segment(x_train['text'].values,max_seq_length)
x_test_mask,x_test_segment,x_test_tokens = mask_and_segment(x_test['text'].values,max_seq_length)

BERT MODEL

In [25]:
pip install tensorflow_hub

In [26]:
import tensorflow_hub as hub
from tensorflow.keras.models import Model

In [27]:
#Load the pretrained model from tensorflow hub
tf.keras.backend.clear_session()

# We are taking the maximum length of a sequence

max_seq_length=55

#BERT takes 3 input

input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,),dtype = tf.int32,name = "input_word_ids")

input_mask = tf.keras.layers.Input(shape=(max_seq_length,),dtype=tf.int32, name ="input mask")
#second seq segment vector are 1's
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="segment_ids")
#bert layer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
trainable=False)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

bert_model = Model(inputs=[input_word_ids,input_mask,segment_ids],outputs= pooled_output)


In [28]:
bert_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 55)]         0           []                               
                                                                                                  
 input mask (InputLayer)        [(None, 55)]         0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 55)]         0           []                               
                                                                                                  
 keras_layer (KerasLayer)       [(None, 768),        109482241   ['input_word_ids[0][0]',         
                                 (None, 55, 768)]                 'input mask[0][0]',         

In [29]:
bert_model.input

[<KerasTensor: shape=(None, 55) dtype=int32 (created by layer 'input_word_ids')>,
 <KerasTensor: shape=(None, 55) dtype=int32 (created by layer 'input mask')>,
 <KerasTensor: shape=(None, 55) dtype=int32 (created by layer 'segment_ids')>]

In [30]:
bert_model.output

<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'keras_layer')>

In [31]:
x_train_pooled_output = bert_model.predict([x_train_tokens,x_train_mask,x_train_segment])

In [32]:
x_test_pooled_output = bert_model.predict([x_test_tokens,x_test_mask,x_test_segment])

Creating the NN network

In [33]:
from tensorflow.keras.layers import Input,Dense,Activation,Dropout
from tensorflow.keras.models import Model

In [36]:
import warnings
warnings.filterwarnings("ignore")

input_layer = Input(shape=(768,))
#hidden layer
layer1 = Dense(64,activation='relu')(input_layer)
#hidden layer2
layer2 = Dense(32,activation='relu')(layer1)
#hidden layer3
layer3 = Dense(16,activation='relu')(layer2)
#Dense hidde layer
output = Dense(1,activation='sigmoid')(layer3)
#creating the model
model = Model(inputs=input_layer,outputs=output)
model.compile(loss ='binary_crossentropy',optimizer='adam',metrics=['AUC'])
model.build(768)
model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 768)]             0         
                                                                 
 dense_8 (Dense)             (None, 64)                49216     
                                                                 
 dense_9 (Dense)             (None, 32)                2080      
                                                                 
 dense_10 (Dense)            (None, 16)                528       
                                                                 
 dense_11 (Dense)            (None, 1)                 17        
                                                                 
Total params: 51,841
Trainable params: 51,841
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.fit(x_train_pooled_output,y_train,batch_size=32, epochs=50,validation_data=(x_test_pooled_output,y_test))

Epoch 1/50
660/660 [==============================] - 3s 4ms/step - loss: 0.3343 - auc: 0.8630 - val_loss: 0.3444 - val_auc: 0.8555
Epoch 2/50
660/660 [==============================] - 2s 3ms/step - loss: 0.3292 - auc: 0.8681 - val_loss: 0.3446 - val_auc: 0.8588
Epoch 3/50
660/660 [==============================] - 2s 3ms/step - loss: 0.3297 - auc: 0.8662 - val_loss: 0.3851 - val_auc: 0.8606
Epoch 4/50
660/660 [==============================] - 2s 3ms/step - loss: 0.3299 - auc: 0.8675 - val_loss: 0.3415 - val_auc: 0.8563
Epoch 5/50
660/660 [==============================] - 2s 3ms/step - loss: 0.3266 - auc: 0.8717 - val_loss: 0.3345 - val_auc: 0.8636
Epoch 6/50
660/660 [==============================] - 2s 3ms/step - loss: 0.3320 - auc: 0.8652 - val_loss: 0.3377 - val_auc: 0.8629
Epoch 7/50
660/660 [==============================] - 2s 3ms/step - loss: 0.3331 - auc: 0.8647 - val_loss: 0.3428 - val_auc: 0.8592
Epoch 8/50
660/660 [==============================] - 2s 3ms/step - loss: 0.

In [51]:
from tensorflow.keras.models import load_model
model.save('saved_model/my_model.h5') 

In [58]:
from tensorflow.keras.models import load_model
bert_model.save('saved_model/bert_model.h5') 

In [54]:
NN_model = tf.keras.models.load_model('saved_model/my_model.h5')

# Show the model architecture
NN_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 768)]             0         
                                                                 
 dense_8 (Dense)             (None, 64)                49216     
                                                                 
 dense_9 (Dense)             (None, 32)                2080      
                                                                 
 dense_10 (Dense)            (None, 16)                528       
                                                                 
 dense_11 (Dense)            (None, 1)                 17        
                                                                 
Total params: 51,841
Trainable params: 51,841
Non-trainable params: 0
_________________________________________________________________


In [57]:
y_test_pred = NN_model.predict(x_test_pooled_output)
print(y_test_pred)

[[0.55986863]
 [0.9723655 ]
 [0.85925055]
 ...
 [0.91751146]
 [0.874603  ]
 [0.9549502 ]]
